In [5]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler


In [25]:
# Carregar os datasets de treinamento e teste
ingredientes = pd.read_csv('./datasets/preprocessed/INGREDIENTES_normalizados_processed.csv')
pratosxingredientes_train = pd.read_csv('./datasets/train/PRATOSxINGREDIENTES_treinamento.csv')


In [27]:
# criando um DataFrame com zeros para os sabores
todos_sabores = [col for col in ingredientes.columns if col not in ['ingrediente', 'categoria']]
sabores_zeros_df = pd.DataFrame(0.0, index=pratosxingredientes_train.index, columns=todos_sabores)

In [29]:
sabores_zeros_df

,nutty,peanut,cocoa,musty,roasted,hazelnut,roast,earthy,potato,woody,...,sandalwood,green tea,patchouli,tangy,pleasant,cardboard,maple syrup,skunky,tomato leaf,outdoor
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
65,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
66,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
67,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
# concatenando o DataFrame de sabores com o DataFrame de pratos
pratosxingredientes_train = pd.concat([pratosxingredientes_train, sabores_zeros_df], axis=1)

In [33]:
# função para somar os sabores dos ingredientes
def somar_sabores_por_prato(ingredientes_prato):
    soma_sabores = pd.Series(0, index=todos_sabores, dtype=float)
    
    for ingrediente in ingredientes_prato:
        row = ingredientes[ingredientes['ingrediente'] == ingrediente]
        
        if not row.empty:
            valores_numericos = row.iloc[0, 2:].astype(float)  # Ignorar as duas primeiras colunas
            soma_sabores += valores_numericos
    
    return soma_sabores

# aplicando a função de soma dos sabores para cada prato
for index, row in pratosxingredientes_train.iterrows():
    ingredientes_prato = eval(row['ingredientes'])  # Converter a string em uma lista
    sabores_soma = somar_sabores_por_prato(ingredientes_prato)
    
    for sabor in todos_sabores:
        if sabor in pratosxingredientes_train.columns:
            pratosxingredientes_train.loc[index, sabor] = sabores_soma[sabor]

In [35]:
pratosxingredientes_train

,ingredientes,pratos_allrecipes,prato,nutty,peanut,cocoa,musty,roasted,hazelnut,roast,...,sandalwood,green tea,patchouli,tangy,pleasant,cardboard,maple syrup,skunky,tomato leaf,outdoor
0,"{'apple', 'frog fish and sea devils', 'sugar',...",caramel apple pie cookies,Apple pie,0.292683,0.538462,0.388889,0.565217,0.043478,0.090909,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"{'sugar', 'vanilla', 'ananas', 'peanut butter'...",banana and peanut butter 4 ingredient ice cream,Ice cream,0.780488,0.615385,0.777778,1.000000,0.260870,0.454545,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"{'cinnamon', 'vanilla', 'milk powder', 'ananas...",banana cinnamon chocolate sorbet,Chocolate,1.024390,0.923077,1.166667,1.434783,0.304348,0.454545,0.083333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"{'honey', 'almond', 'white-flowered gourd', 's...",flourless peanut butter chocolate chip blender...,Chocolate,0.756098,1.000000,0.777778,1.000000,0.391304,0.636364,0.250000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"{'avocado', 'ananas', 'fatty fish', 'cocoa'}",abc pudding avocado banana chocolate delight,Chocolate,1.439024,1.461538,1.500000,1.695652,0.739130,1.090909,0.750000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,"{'pepper', 'cheddar cheese', 'water', 'corn oi...",jazzy stuffed peppers,Stuffed peppers,0.951220,1.000000,1.111111,1.130435,0.173913,0.545455,0.083333,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
65,"{'cheese', 'butter', 'cream cheese', 'milk', '...",no bake sugar free strawberry cheesecake,Strawberry,1.073171,1.230769,1.611111,1.478261,0.217391,0.636364,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
66,"{'sugar', 'vanilla', 'milk powder', 'milk', 'c...",coffee and donuts tiramisu,Tiramisu,1.487805,1.230769,1.500000,1.391304,1.130435,0.909091,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
67,"{'sugar', 'milk powder', 'syrup', 'rum', 'choc...",chocolate rum snowballs,Chocolate,0.682927,0.769231,0.888889,0.782609,0.130435,0.272727,0.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
pratosxingredientes_train[['ingredientes','pratos_allrecipes','prato','sour', 'bitter', 'sweet', 'fruity', 'grassy','roast', 'caramel', 'fermented','powdery','dry']]

,ingredientes,pratos_allrecipes,prato,sour,bitter,sweet,fruity,grassy,roast,caramel,fermented,powdery,dry
0,"{'apple', 'frog fish and sea devils', 'sugar',...",caramel apple pie cookies,Apple pie,1.153846,0.896104,0.921260,1.000000,0.750,0.000000,0.30,0.500000,0.571429,0.500000
1,"{'sugar', 'vanilla', 'ananas', 'peanut butter'...",banana and peanut butter 4 ingredient ice cream,Ice cream,2.153846,1.415584,1.543307,1.415842,1.000,0.000000,0.80,0.833333,1.285714,0.916667
2,"{'cinnamon', 'vanilla', 'milk powder', 'ananas...",banana cinnamon chocolate sorbet,Chocolate,2.846154,2.077922,2.165354,1.702970,1.625,0.083333,1.05,1.166667,2.285714,1.916667
3,"{'honey', 'almond', 'white-flowered gourd', 's...",flourless peanut butter chocolate chip blender...,Chocolate,2.000000,1.961039,1.425197,1.297030,1.500,0.250000,0.60,0.833333,1.714286,1.000000
4,"{'avocado', 'ananas', 'fatty fish', 'cocoa'}",abc pudding avocado banana chocolate delight,Chocolate,2.461538,1.987013,1.937008,1.524752,1.125,0.750000,1.00,1.166667,1.571429,1.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,"{'pepper', 'cheddar cheese', 'water', 'corn oi...",jazzy stuffed peppers,Stuffed peppers,2.615385,1.649351,1.803150,1.297030,1.500,0.083333,0.85,1.166667,1.571429,1.583333
65,"{'cheese', 'butter', 'cream cheese', 'milk', '...",no bake sugar free strawberry cheesecake,Strawberry,3.076923,1.350649,1.929134,2.158416,1.000,0.000000,1.25,2.000000,0.428571,0.583333
66,"{'sugar', 'vanilla', 'milk powder', 'milk', 'c...",coffee and donuts tiramisu,Tiramisu,2.153846,1.207792,1.417323,1.267327,1.000,1.000000,1.45,0.500000,1.571429,1.083333
67,"{'sugar', 'milk powder', 'syrup', 'rum', 'choc...",chocolate rum snowballs,Chocolate,1.769231,1.077922,1.149606,1.089109,1.125,0.333333,0.45,0.833333,1.571429,0.750000


In [46]:
# inicializando o normalizador MinMaxScaler
scaler = MinMaxScaler()

pratosxingredientes_train[todos_sabores] = scaler.fit_transform(pratosxingredientes_train[todos_sabores])

In [48]:
pratosxingredientes_train[['ingredientes','pratos_allrecipes','prato','sour', 'bitter', 'sweet', 'fruity', 'grassy','roast', 'caramel', 'fermented','powdery','dry']]

,ingredientes,pratos_allrecipes,prato,sour,bitter,sweet,fruity,grassy,roast,caramel,fermented,powdery,dry
0,"{'apple', 'frog fish and sea devils', 'sugar',...",caramel apple pie cookies,Apple pie,0.188406,0.166247,0.202773,0.355634,0.129032,0.000000,0.133333,0.176471,0.12500,0.115385
1,"{'sugar', 'vanilla', 'ananas', 'peanut butter'...",banana and peanut butter 4 ingredient ice cream,Ice cream,0.376812,0.267003,0.339688,0.503521,0.193548,0.000000,0.355556,0.294118,0.28125,0.211538
2,"{'cinnamon', 'vanilla', 'milk powder', 'ananas...",banana cinnamon chocolate sorbet,Chocolate,0.507246,0.395466,0.476603,0.605634,0.354839,0.076923,0.466667,0.411765,0.50000,0.442308
3,"{'honey', 'almond', 'white-flowered gourd', 's...",flourless peanut butter chocolate chip blender...,Chocolate,0.347826,0.372796,0.313692,0.461268,0.322581,0.230769,0.266667,0.294118,0.37500,0.230769
4,"{'avocado', 'ananas', 'fatty fish', 'cocoa'}",abc pudding avocado banana chocolate delight,Chocolate,0.434783,0.377834,0.426343,0.542254,0.225806,0.692308,0.444444,0.411765,0.34375,0.307692
...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,"{'pepper', 'cheddar cheese', 'water', 'corn oi...",jazzy stuffed peppers,Stuffed peppers,0.463768,0.312343,0.396880,0.461268,0.322581,0.076923,0.377778,0.411765,0.34375,0.365385
65,"{'cheese', 'butter', 'cream cheese', 'milk', '...",no bake sugar free strawberry cheesecake,Strawberry,0.550725,0.254408,0.424610,0.767606,0.193548,0.000000,0.555556,0.705882,0.09375,0.134615
66,"{'sugar', 'vanilla', 'milk powder', 'milk', 'c...",coffee and donuts tiramisu,Tiramisu,0.376812,0.226700,0.311958,0.450704,0.193548,0.923077,0.644444,0.176471,0.34375,0.250000
67,"{'sugar', 'milk powder', 'syrup', 'rum', 'choc...",chocolate rum snowballs,Chocolate,0.304348,0.201511,0.253033,0.387324,0.225806,0.307692,0.200000,0.294118,0.34375,0.173077


In [23]:
# exportando base de pratos tratada

pratosxingredientes_train.to_csv('./datasets/preprocessed/PRATOS_processed.csv', index=False)